# Generate pressure coefficients

Initialize Path Manager

In [1]:
from cfdmod.use_cases.pressure.path_manager import CpPathManager
import pathlib

path_manager = CpPathManager(output_path=pathlib.Path("./output/pressure"))

Read post-processing Cp config file

In [2]:
from cfdmod.use_cases.pressure.cp_config import CpCaseConfig

cfg_path = pathlib.Path("./fixtures/tests/pressure/cp_params.yaml")
post_proc_cfg = CpCaseConfig.from_file(cfg_path)

post_proc_cfg

CpCaseConfig(pressure_coefficient={'default': CpConfig(statistics=[BasicStatisticModel(stats='mean', display_name=''), BasicStatisticModel(stats='rms', display_name=''), BasicStatisticModel(stats='skewness', display_name=''), BasicStatisticModel(stats='kurtosis', display_name=''), ParameterizedStatisticModel(stats='mean_eq', display_name='', params=MeanEquivalentParamsModel(scale_factor=0.61)), ParameterizedStatisticModel(stats='min', display_name='', params=ExtremeAbsoluteParamsModel(method_type='Absolute')), ParameterizedStatisticModel(stats='max', display_name='', params=ExtremeGumbelParamsModel(method_type='Gumbel', peak_duration=3.0, event_duration=600.0, n_subdivisions=10, non_exceedance_probability=0.78, full_scale_U_H=40.0, full_scale_characteristic_length=22.4))], number_of_chunks=10, timestep_range=(10000.0, 20000.0), reference_pressure='average', simul_U_H=0.05, simul_characteristic_length=0.8445)})

Read LNAS Mesh

In [3]:
from lnas import LnasFormat

mesh_path = pathlib.Path("./fixtures/tests/pressure/galpao/galpao.lnas")
mesh = LnasFormat.from_file(mesh_path)

mesh.geometry.vertices.shape, mesh.geometry.triangles.shape

((1549, 3), (2915, 3))

Read hist series

In [4]:
static_data_path = pathlib.Path(
    "./fixtures/tests/pressure/data/new.points.static_pressure.data.resampled.h5"
)
body_data_path = pathlib.Path("./fixtures/tests/pressure/data/new.bodies.galpao.data.resampled.h5")

Process pressure coefficient use case

In [5]:
import shutil
from cfdmod.logger import logger
from cfdmod.use_cases.pressure.cp_data import process_cp
from cfdmod.use_cases.pressure.path_manager import copy_input_artifacts

for cfg_lbl, cfg in post_proc_cfg.pressure_coefficient.items():
    logger.info(f"Processing pressure coefficients for config {cfg_lbl} ...")
    process_cp(
        pressure_data_path=static_data_path,
        body_data_path=body_data_path,
        cfg_label=cfg_lbl,
        cfg=cfg,
        mesh=mesh.geometry,
        path_manager=path_manager,
    )

    logger.info("Copying input artifacts")
    shutil.copy(cfg_path, path_manager.get_config_path(cfg_lbl))
    copy_input_artifacts(
        cfg_path=cfg_path,
        mesh_path=mesh_path,
        static_data_path=static_data_path,
        body_data_path=body_data_path,
        path_manager=path_manager,
    )

    logger.info(f"Processed pressure coefficients for config {cfg_lbl}!")

[2024-05-23 12:03:27,727] [INFO] - cfdmod - Processing pressure coefficients for config default ... (739055453.py:7)
[2024-05-23 12:03:27,728] [INFO] - cfdmod - Transforming into pressure coefficient (cp_data.py:184)


/home/ubuntu/Documentos/Repositories/cfdmod/cfdmod/use_cases/pressure/cp_data.py:178: RuntimeWarning: Path for time series already exists output/pressure/cp/default/cp.time_series.h5. Deleted old file
  warnings.warn(


[2024-05-23 12:03:31,568] [INFO] - cfdmod - Dividing into point groups (cp_data.py:201)


/home/ubuntu/Documentos/Repositories/cfdmod/cfdmod/use_cases/pressure/cp_data.py:195: RuntimeWarning: Path for grouped time series already exists output/pressure/cp/default/time_series.grouped.h5. Deleted old file
  warnings.warn(


[2024-05-23 12:03:32,454] [INFO] - cfdmod - Calculating statistics (cp_data.py:208)
[2024-05-23 12:03:33,522] [INFO] - cfdmod - Copying input artifacts (739055453.py:17)
[2024-05-23 12:03:33,526] [INFO] - cfdmod - Processed pressure coefficients for config default! (739055453.py:27)
